In [4]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import pandas as pd

In [5]:
def extract_info(item):
    v_info = {"min_salary" : 0,
              "max_salary" : 0,
              "currency" : '', 
              "title": '', 
              "link":'',
              "source":''}
    # Название вакансии
    title = item.find_all('a', {'data-qa':'vacancy-serp__vacancy-title'})[0].text
    v_info['title'] = title

    # Зарплата
    money1 = item.find_all('span',{'data-qa':['vacancy-serp__vacancy-compensation']})[0].text

    money1 = money1.replace('\u202f','')

    if 'руб' in money1: 
        money1 = money1.replace(' руб.','')
        v_info["currency"] = 'руб'

    if '–' in money1: 
        money1 = money1.split(' – ')
        v_info["min_salary"] = int(money1[0])
        v_info["max_salary"] = int(money1[1])

    if 'от' in money1: 
        money1 = money1.replace('от ','')
        v_info["min_salary"] = int(money1)
    
    #Ссылка
    v_info['link'] = item.find_all('a')[0].get('href')
    
    #Источник
    v_info["source"] = 'hh.ru'
    return v_info

In [6]:
headers = {'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

occupation = 'учитель'
main_link = 'https://hh.ru'

response = requests.get(f'{main_link}/search/vacancy?area=1&st=searchVacancy&text={occupation}', headers=headers)
dom = BeautifulSoup(response.text, 'html.parser')

In [7]:
# Находим кол-во страниц

div = dom.find_all('div', {'class': ['pager']})[0]
a_bb = div.find_all('a', {'class':'bloko-button'})

pages = []
for b in a_bb:
    pages.append(b.find_all('span')[0].text)

num_of_pages = int(pages[-2])
print(num_of_pages)

40


In [8]:
# Скачиваем данные со страниц
vacs = []
num_of_pages = 3
for page in range(num_of_pages):
    print("Загрузка страницы: ", page)
    link = f'{main_link}/search/vacancy?area=1&st=searchVacancy&text={occupation}&page={page}'
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.text, 'html.parser')
    vacs += dom.find_all('div', {'class': ['vacancy-serp-item']})
#     soup = bs(response,'lxml')
#     vacancy += (soup.find_all('div', {'data-qa':'vacancy-serp__vacancy'}))

print("Всего вакансий: ", len(vacs))

Загрузка страницы:  0
Загрузка страницы:  1
Загрузка страницы:  2
Всего вакансий:  60


In [9]:
# Обрабатываем вакансии

vs_info = []
for item in vacs:
    try:
        vs_info.append(extract_info(item))
    except:
        pass

In [10]:
# Cмотрим результат
df_hh = pd.DataFrame(vs_info)
df_hh.head()

,min_salary,max_salary,currency,title,link,source
0,40000,100000,руб,Педагог начальных классов со знанием английско...,https://nn.hh.ru/vacancy/67452288?from=vacancy...,hh.ru
1,40000,100000,руб,Преподаватель ментальной арифметики,https://nn.hh.ru/vacancy/67452266?from=vacancy...,hh.ru
2,80000,0,руб,Учитель иностранного языка в школу,https://nn.hh.ru/vacancy/67622589?from=vacancy...,hh.ru
3,120000,0,руб,Учитель начальных классов,https://nn.hh.ru/vacancy/52238064?from=vacancy...,hh.ru
4,95000,105000,руб,Учитель русского языка и литературы,https://nn.hh.ru/vacancy/67628467?from=vacancy...,hh.ru


In [190]:
#Сохраняем данные в файл
df_hh.to_csv(f'HeadHunter.csv')